In [1]:
from IPython import parallel
clients = parallel.Client()
view = clients.load_balanced_view()
print("Number of clients: {}".format(len(clients)))

#with clients.direct_view().sync_imports():
#    import pymc3

Number of clients: 4


In [6]:
%%px
import pymc3

In [7]:
def linear_fit(m):
    SAMPLES = 100
    SIGMA = 0.2
    x = np.random.rand(SAMPLES)
    y = m * x + SIGMA * np.random.randn(SAMPLES)
    data = dict(x=x, y=y)
    
    with pymc3.Model() as model:
        pymc3.glm.glm('y ~ x', data)
        step = pymc3.Slice()
        trace = pymc3.sample(2000, step)
    
    return np.mean(trace['x'])

In [8]:
res = view.map_sync(linear_fit, range(8))

In [9]:
res

[-0.1039286445316695,
 0.96829167578668651,
 1.914161697333854,
 2.9984942623350377,
 3.9964351830454024,
 5.0581820369063557,
 6.0159619324304989,
 7.027377403761589]